<a href="https://colab.research.google.com/github/manassanant132/02_Maching_Learing_Pipeline/blob/main/MAIN_FILE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0.MOUNT

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1.DATA PREPARATION

In [10]:
import tensorflow as tf 
import time
import numpy as np
import matplotlib.pyplot as plt

import plotly
import plotly.graph_objs as go

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle as p

to_categorical = tf.keras.utils.to_categorical  #เอา tenflow แยกประเภท
mnist = tf.keras.datasets.mnist  #download data set ตัวเลข(mnist)
load_model = tf.keras.models.load_model  
model_from_jason = tf.keras.models.model_from_json  # model -> json

In [11]:
batch_size = 128
num_classes = 10
epochs =5

img_rows,img_cols = 28,28

In [12]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [13]:
temp_x_test = x_test

x_train = x_train.reshape(x_train.shape[0], img_rows,img_cols,1)
x_test = x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
input_shape = (img_rows,img_cols,1)

x_train = x_train.astype('float32')
x_test=x_test.astype('float32')
x_train /=255
x_test /=255
print('x_train.shape:', x_train.shape)
print('x_test.shape:', x_test.shape)
print(x_train.min(), x_train.max())

x_train.shape: (60000, 28, 28, 1)
x_test.shape: (10000, 28, 28, 1)
0.0 1.0


In [14]:
y_train= to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)

# DEFINE MODEL

In [15]:
#Feature Extraction
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape))
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

#Image Classification
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes,activation='softmax'))

In [16]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 128)               1179776   
                                                        

In [17]:
start = time.time()

his = model.fit(x_train,y_train, batch_size=batch_size , epochs=epochs ,verbose=1, validation_data=(x_test,y_test))

done = time.time()
print(done-start)

Epoch 1/5
469/469 [==============================] - 129s 273ms/step - loss: 0.2452 - accuracy: 0.9268 - val_loss: 0.0502 - val_accuracy: 0.9814
Epoch 2/5
469/469 [==============================] - 129s 274ms/step - loss: 0.0853 - accuracy: 0.9747 - val_loss: 0.0371 - val_accuracy: 0.9873
Epoch 3/5
469/469 [==============================] - 129s 275ms/step - loss: 0.0625 - accuracy: 0.9815 - val_loss: 0.0305 - val_accuracy: 0.9896
Epoch 4/5
469/469 [==============================] - 129s 275ms/step - loss: 0.0531 - accuracy: 0.9836 - val_loss: 0.0288 - val_accuracy: 0.9912
Epoch 5/5
469/469 [==============================] - 128s 273ms/step - loss: 0.0443 - accuracy: 0.9868 - val_loss: 0.0262 - val_accuracy: 0.9913
682.6116104125977


# Save Model

## 1.save history

In [18]:
filepath_history_model = 'history_model'

with open(filepath_history_model,'wb') as file :
  p.dump(his.history,file)

In [23]:
filepath='model.h5'
model.save(filepath)

In [24]:
filepath_model='model.json'
filepath_weights = 'weights_model.h5'

model_json = model.to_json()
with open(filepath_model,"w") as json_file :
  json_file.write(model_json)
  model.save_weights(filepath_weights)